In [4]:
import cv2
from ultralytics import YOLO

# Load your trained YOLOv8m model
model = YOLO("C:\\Users\\dhaks\\Downloads\\Pallet detection\\best.pt")  # <-- change this to your model path

# Load the image you want to test
image_path = "C:\\Users\\dhaks\\Downloads\\Pallet detection\\sample-2.jpg"  # <-- change this to your image
image = cv2.imread(image_path)

# Run inference
results = model(image)

# Initialize some values
docking_side_detected = False

# Draw results and check aspect ratio
for result in results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Only process if confidence score is above a threshold (e.g., 0.5)
        if conf < 0.5:
            continue

        # Calculate width and height of the bounding box
        width = x2 - x1
        height = y2 - y1

        aspect_ratio = width / height

        # Dynamic threshold for aspect ratio, adjusting for different pallet orientations
        if aspect_ratio > 1.5:  # Adjust this threshold based on your observation
            text = "Possible Docking side detected"
            color = (0, 255, 0)  # Green
            docking_side_detected = True
        else:
            text = "Not docking side"
            color = (0, 0, 255)  # Red

        # Draw box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{text} | AR: {aspect_ratio:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)  # Increased font size and thickness

# Add a fallback check (if no docking side detected, you may instruct AMR to move)
if not docking_side_detected:
    print("No docking side detected. Instruct AMR to move left/right and search further.")

# Resize image to fit your screen (Optional, helps if the image is too large)
resized_image = cv2.resize(image, (800, 600))

# Show the result: This will display the entire image with bounding boxes and labels.
cv2.imshow("Docking Side Detection", resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 480x640 5 Pallets, 657.1ms
Speed: 7.4ms preprocess, 657.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
